### Лабораторная работа №7
Выполнила Карасева Екатерина M33351

#### Обработка последовательностей

Обработка текста

In [1]:
import re


def clearing_file(input_filename, output_filename):
    with open(input_filename, 'r', encoding='utf-8') as input_file:
        with open(output_filename, 'w', encoding='utf-8') as output_file:
            text = input_file.read()
            text = text.lower()
            text = text.replace('ё', 'е')
            text = text.replace('!', '.')
            text = text.replace('?', '.')
            text = ''.join(ch if bool(re.search('[а-я. \n]', ch)) else '' for ch in text)
            text = re.sub(r'\s+', r' ', text)
            text = re.sub(r' *\n *', r'', text)
            text = re.sub(r'\n\.\n', r'', text)
            text = re.sub(r'\.+', r'.', text)

            output_file.write(text)
            return text


file = 'text.txt'
clear_file = 'clear_text.txt'

clear_file_text = clearing_file(file, clear_file)

In [2]:
to_one_hot = {}
from_one_hot = {}
a = ord('а')
len_d = 34
ind = 0
for litter in [chr(i) for i in range(a, a + len_d - 2)] + [' ', '.']:
    l = [0] * len_d
    l[ind] = 1
    to_one_hot[litter] = l
    from_one_hot[str(l)] = litter
    ind += 1


def get_one_hot(text):
    return list(map(lambda ch: to_one_hot[ch], text))

In [3]:
from nltk import ngrams
import numpy as np

len_ngrams = 10


def get_ngrams(arr, n):
    ngrams_text = list(ngrams(arr, n + 1))
    size_ngrams = len(ngrams_text)
    x = np.empty(shape=[size_ngrams, n, len_d])
    y = np.empty(shape=[size_ngrams, len_d])
    for window, i in zip(ngrams_text, range(size_ngrams)):
        x[i] = np.array(window[:n])
        y[i] = np.array(window[-1])
    return x, y


In [4]:
from tensorflow import keras
from keras import layers

obj, target = get_ngrams(get_one_hot(clear_file_text), len_ngrams)

model = keras.Sequential()
model.add(keras.Input(shape=obj.shape[1:]))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.LSTM(256))
model.add(layers.Dense(target.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(obj, target, batch_size=128, epochs=30)

Epoch 1/30
117/117 [==============================] - 16s 112ms/step - loss: 3.1020 - accuracy: 0.1618

In [5]:
def random_string_generating(filename, predict):
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
        sentences = text.split('.')
        sentences = list(filter(lambda x: len(x) >= len_ngrams, sentences))
        sentences_count = len(sentences)

        index = np.random.randint(0, sentences_count - 1)

        text = sentences[index][:len_ngrams]
        result_text = text
        cur_text = text

        for i in range(100):
            prediction = predict(cur_text)
            result_text += prediction
            cur_text = cur_text[1:] + prediction

        return sentences[index], text, result_text

def lstm_predict(text):
    one_hot = get_one_hot(text)
    test = [one_hot]
    prediction = np.argmax(model.predict(test))
    one_hot_pred = [0] * len_d
    one_hot_pred[prediction] = 1
    return from_one_hot[str(one_hot_pred)]

In [18]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, lstm_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

1/1 [==============================] - 0s 31ms/step
 голос младшего почти не дрожит
 голос мла
 голос младшего из объятия. тот обнимает крепко и продолжает поглаживания по своей спине знает что будет когда


In [7]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, lstm_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

1/1 [==============================] - 0s 31ms/step
 единственное что еще помогает не сойти с ума в этой вечной апатии это мысли
 единствен
 единственный смысл жить сейчас. кажется все мои цели и мечты всегда были связаны с тобой. может тут нужен инг


In [20]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, lstm_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

1/1 [==============================] - 0s 31ms/step
 это наверное будет последней стадией отчаяния
 это навер
 это наверное поэтому ты и оставил меня одного кататься на этих долбанных эмоциональных американских горках. о


### Марковские цепи

In [9]:
from collections import defaultdict

def data_to_markov(data):
    ngrams_text = list(ngrams(data, len_ngrams + 1))
    size_ngrams = len(ngrams_text)

    x = []
    y = []
    for window, index in zip(ngrams_text, range(size_ngrams)):
        x.append(''.join(window[:len_ngrams]))
        y.append(window[-1])

    return np.array(x), np.array(y)

def get_markov(x_train, y_train):
    nodes_markov = defaultdict(lambda: defaultdict(lambda: 0))
    for sentence, symbol in zip(x_train, y_train):
        nodes_markov[sentence][symbol] += 1
    return nodes_markov

x_markov, y_markov = data_to_markov(clear_file_text)
nodes = get_markov(x_markov, y_markov)

In [10]:
def markov_predict(text):
    node = nodes[text]
    weights = np.array([w for w in node.values()])
    if len(weights) == 0:
        return '.'
    index = node[np.argmax(weights)]
    return [w for w in node][index]

full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, markov_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

 единственное что еще помогает не сойти с ума в этой вечной апатии это мысли
 единствен
 единственный смысл жить сейчас. кажется все мои цели и мечты всегда были связаны с тобой. если ты полностью и


In [11]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, markov_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

 пораженный юнги медленно стараясь не напугать парня обхватывает того за талию и подтягивает ближе к себе
 пораженны
 пораженный юнги медленно смакуя вкус боли и обиды. разбиваться тоже постепенно даже скорее крошиться в пыль ч


In [12]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, markov_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

 ненависть к себе
 ненависть
 ненависть к таким сопливым произведениям о вечной любви. вечность. как оказалось ничего в этом мире вечного н
